In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid
from sklearn.metrics import confusion_matrix

#SMOTE
from imblearn.over_sampling import SMOTE

#Mô hình XG Boost
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

#Random Forest
from sklearn.ensemble import RandomForestClassifier

#Light GBM
!pip install lightgbm
from lightgbm import LGBMClassifier

In [3]:
data = pd.read_csv('/content/drive/MyDrive/Datasets/Home-credit-default-risk/new/application_train_(new_data).csv')
data

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,...,FLAG_DOCUMENT_11,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,1,Cash loans,M,N,0,202500.0,406597.5,351000.0,Unaccompanied,Working,...,0,0,0,0,0,0,0,0.0,0.0,1.0
1,0,Cash loans,F,N,0,270000.0,1293502.5,1129500.0,Family,State servant,...,0,0,0,0,0,0,0,0.0,0.0,0.0
2,0,Revolving loans,M,Y,0,67500.0,135000.0,135000.0,Unaccompanied,Working,...,0,0,0,0,0,0,0,0.0,0.0,0.0
3,0,Cash loans,M,N,0,121500.0,513000.0,513000.0,Unaccompanied,Working,...,0,0,0,0,0,0,0,0.0,0.0,0.0
4,0,Cash loans,M,N,0,99000.0,490495.5,454500.0,"Spouse, partner",State servant,...,0,0,0,0,0,0,0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216147,0,Cash loans,F,N,1,94500.0,225000.0,225000.0,Unaccompanied,Working,...,0,0,0,0,0,0,0,0.0,0.0,1.0
216148,0,Cash loans,F,N,0,112500.0,345510.0,247500.0,Unaccompanied,Commercial associate,...,0,0,0,0,0,0,0,1.0,0.0,2.0
216149,0,Cash loans,F,N,0,153000.0,677664.0,585000.0,Unaccompanied,Working,...,0,0,0,0,0,0,0,1.0,0.0,1.0
216150,1,Cash loans,F,N,0,171000.0,370107.0,319500.0,Unaccompanied,Commercial associate,...,0,0,0,0,0,0,0,0.0,0.0,0.0


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216152 entries, 0 to 216151
Data columns (total 59 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   TARGET                        216152 non-null  int64  
 1   NAME_CONTRACT_TYPE            216152 non-null  object 
 2   CODE_GENDER                   216152 non-null  object 
 3   FLAG_OWN_CAR                  216152 non-null  object 
 4   CNT_CHILDREN                  216152 non-null  int64  
 5   AMT_INCOME_TOTAL              216152 non-null  float64
 6   AMT_CREDIT                    216152 non-null  float64
 7   AMT_GOODS_PRICE               216152 non-null  float64
 8   NAME_TYPE_SUITE               216152 non-null  object 
 9   NAME_INCOME_TYPE              216152 non-null  object 
 10  NAME_EDUCATION_TYPE           216152 non-null  object 
 11  NAME_FAMILY_STATUS            216152 non-null  object 
 12  NAME_HOUSING_TYPE             216152 non-nul

In [8]:
obj_cols = data.select_dtypes(include='object').columns
for i in obj_cols:
  print(data[i].value_counts())
  print('\n')

NAME_CONTRACT_TYPE
Cash loans         195058
Revolving loans     21094
Name: count, dtype: int64


CODE_GENDER
F    135632
M     80520
Name: count, dtype: int64


FLAG_OWN_CAR
N    133336
Y     82816
Name: count, dtype: int64


NAME_TYPE_SUITE
Unaccompanied      176883
Family              27145
Spouse, partner      8163
Children             1941
Other_B              1218
Other_A               621
Group of people       181
Name: count, dtype: int64


NAME_INCOME_TYPE
Working                 135697
Commercial associate     61109
State servant            19320
Student                     14
Pensioner                    7
Businessman                  4
Maternity leave              1
Name: count, dtype: int64


NAME_EDUCATION_TYPE
Secondary / secondary special    147620
Higher education                  58457
Incomplete higher                  8069
Lower secondary                    1884
Academic degree                     122
Name: count, dtype: int64


NAME_FAMILY_STATUS
Married          

In [4]:
#Label encoding có các columns:
def label_encoding(data):
  for i in ['NAME_CONTRACT_TYPE', 'CODE_GENDER', 'FLAG_OWN_CAR', 'HOUSETYPE_MODE']:
    value_counts = data[i].value_counts().index
    value_to_label = {value: idx for idx, value in enumerate(value_counts)}
    data[i] = data[i].map(value_to_label)
  return data

In [5]:
#Hàm label encoding cho các columns còn lại:
def mapping_cols(data, columns_names = ['NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
                                       'NAME_HOUSING_TYPE', 'OCCUPATION_TYPE', 'WALLSMATERIAL_MODE']):
  NAME_TYPE_SUITE_mapping = {'Unaccompanied': 0, 'Family': 1, 'Spouse, partner': 1, 'Children': 1, 'Other_A': 2, 'Other_B': 2, 'Group of people': 2}
  NAME_INCOME_TYPE_mapping = {'Student': 0, 'Maternity leave': 1, 'Pensioner': 2, 'State servant': 3,
                              'Working': 4, 'Commercial associate': 5, 'Businessman': 6}
  NAME_EDUCATION_TYPE_mapping = {'Lower secondary': 0, 'Secondary / secondary special': 1, 'Incomplete higher': 2,
                                 'Higher education': 3, 'Academic degree': 4}
  NAME_FAMILY_STATUS_mapping = {'Single / not married': 0, 'Widow': 1, 'Separated': 2, 'Civil marriage': 3, 'Married': 4}
  NAME_HOUSING_TYPE_mapping = {'With parents': 0, 'Municipal apartment': 1, 'Co-op apartment': 2,
                               'Rented apartment': 3, 'Office apartment': 4, 'House / apartment': 5}
  OCCUPATION_TYPE_mapping = {'Low-skill Laborers': 0, 'Cleaning staff': 0, 'Waiters/barmen staff': 0, 'Laborers': 0, 'Cooking staff': 0,
                             'Private service staff': 0, 'Drivers': 0, 'Security staff': 0, 'Secretaries': 1, 'Sales staff': 1,
                             'HR staff': 1, 'Accountants': 1, 'Core staff': 1, 'Realty agents': 1, 'Medicine staff': 1, 'IT staff': 1,
                             'High skill tech staff': 1, 'Managers': 2}
  WALLSMATERIAL_MODE_mapping = {'Wooden': 0, 'Block': 0, 'Panel': 1, 'Mixed': 1, 'Stone, brick': 2, 'Monolithic': 2, 'Others': 2}

  # Thực hiện ánh xạ cho từng cột trong DataFrame
  mappings = {'NAME_TYPE_SUITE': NAME_TYPE_SUITE_mapping, 'NAME_INCOME_TYPE': NAME_INCOME_TYPE_mapping,
              'NAME_EDUCATION_TYPE': NAME_EDUCATION_TYPE_mapping, 'NAME_FAMILY_STATUS': NAME_FAMILY_STATUS_mapping,
              'NAME_HOUSING_TYPE': NAME_HOUSING_TYPE_mapping, 'OCCUPATION_TYPE': OCCUPATION_TYPE_mapping,
              'WALLSMATERIAL_MODE': WALLSMATERIAL_MODE_mapping}

  for column in columns_names:
    if column in data.columns and column in mappings:
      data[column] = data[column].map(mappings[column])

  return data

In [6]:
data = data.drop(columns = ['ORGANIZATION_TYPE'])
data = label_encoding(data)
data = mapping_cols(data)

***Corr Matrix***

In [7]:
corr = data.corr()
corr.style.background_gradient(cmap='coolwarm')

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_WORK_PHONE,FLAG_PHONE,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,YEARS_BEGINEXPLUATATION_AVG,ELEVATORS_AVG,FLOORSMAX_AVG,YEARS_BEGINEXPLUATATION_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,HOUSETYPE_MODE,WALLSMATERIAL_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_3,FLAG_DOCUMENT_6,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_11,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
TARGET,1.000000,-0.034633,0.046323,-0.030558,0.008044,-0.004290,-0.038587,-0.048050,-0.007523,-0.003324,-0.065683,-0.021676,-0.026323,-0.037497,0.064943,0.073873,0.034575,0.038076,0.017173,-0.023994,-0.050221,0.000339,0.061472,0.063994,-0.027892,0.004287,0.040888,0.041876,0.023602,-0.163132,-0.006295,-0.024520,-0.033295,-0.005781,-0.023011,-0.010570,-0.032780,0.005040,-0.005304,0.012048,0.032710,0.012046,0.030781,0.056128,0.040461,-0.008866,-0.013304,-0.005164,-0.007072,-0.013529,-0.011366,-0.006773,-0.013706,-0.005496,-0.009623,-0.015083,-0.001379,0.023898
NAME_CONTRACT_TYPE,-0.034633,1.000000,-0.016085,-0.000318,0.022844,-0.010823,-0.242060,-0.207113,-0.005744,0.017861,0.053674,-0.040053,-0.012086,0.021300,0.059498,0.033250,0.009268,0.031756,-0.058783,-0.021495,0.031506,0.002237,-0.016407,-0.017545,0.027575,0.010614,0.008564,-0.009985,-0.016872,0.012928,-0.001231,0.009476,0.015409,-0.000134,0.009110,0.000433,0.014518,-0.002688,0.010052,-0.018221,-0.008148,-0.018251,-0.006910,0.064775,-0.568708,-0.028441,-0.095063,-0.007793,0.004788,-0.005405,0.000193,-0.008988,-0.026125,-0.000877,-0.015571,-0.017344,-0.022103,-0.061932
CODE_GENDER,0.046323,-0.016085,1.000000,0.339077,0.015367,0.065238,0.010104,0.011937,-0.009491,0.035505,-0.040589,0.052910,-0.030719,0.015769,0.074651,0.095090,0.049087,-0.055578,-0.004444,-0.017620,-0.209983,0.043308,-0.017679,-0.016833,-0.003722,0.018664,0.037363,0.109861,0.110290,-0.013598,0.003044,0.017419,0.020120,0.002803,0.016068,0.007728,0.019175,-0.005348,-0.007982,-0.009024,-0.017393,-0.008919,-0.015185,0.025475,-0.156854,-0.036047,0.246638,-0.012625,0.001156,0.041309,-0.000858,0.021230,-0.006465,0.003240,0.018170,0.005791,-0.002492,-0.002898
FLAG_OWN_CAR,-0.030558,-0.000318,0.339077,1.000000,0.070602,0.069316,0.099990,0.105329,0.010075,0.028856,0.077252,0.122409,0.025897,0.042642,0.053205,0.033285,0.061182,-0.034062,-0.026682,-0.010150,-0.001158,0.113359,-0.021006,-0.019619,-0.001974,-0.007299,-0.009013,0.047043,0.064433,0.050367,0.005661,0.025878,0.033315,0.004913,0.024767,0.009887,0.032646,-0.008635,-0.018264,0.002922,-0.012959,0.002837,-0.012651,-0.031832,-0.140796,-0.028024,0.221911,-0.016024,-0.002041,0.082760,-0.002350,0.045559,-0.006599,-0.004478,-0.006925,0.017128,-0.006524,-0.026904
CNT_CHILDREN,0.008044,0.022844,0.015367,0.070602,1.000000,-0.003701,-0.019724,-0.023190,0.035031,-0.034229,0.007892,0.197359,0.001061,-0.029771,0.253096,0.067190,0.162937,-0.115719,-0.003551,-0.031700,0.012367,0.894871,0.037793,0.037771,-0.032943,-0.021974,0.000294,0.011351,0.018424,-0.025876,0.006952,-0.007560,-0.010676,0.006285,-0.006824,-0.003820,-0.010253,-0.004077,-0.023672,0.021055,0.004104,0.020748,0.002567,0.002014,-0.008739,-0.042521,0.024408,-0.008792,-0.011173,-0.002805,-0.013049,0.000599,-0.002499,-0.000787,-0.007314,-0.021419,-0.004006,-0.032077
AMT_INCOME_TOTAL,-0.004290,-0.010823,0.065238,0.069316,-0.003701,1.000000,0.134162,0.136612,-0.01395

In [8]:
#Lọc ra các cặp có giá trị corr > 0.5
corr_pairs = corr.unstack().sort_values(ascending=False)
corr_pairs = corr_pairs[corr_pairs < 1]

high_corr_pairs = corr_pairs[corr_pairs > 0.5]
high_corr_pairs = high_corr_pairs.iloc[::2]

In [9]:
#Lọc ra các giá trị corr giữa các biến đầu vào và output
corr_label = corr['TARGET']

In [10]:
#So sánh 2 biến có corr cao, nếu biến nào mà corr của nó với output thấp hơn thì sẽ được loại, ngược lại thì lưu
#Việc này giúp 2 điều: Xóa đi 1 giá trị trong cặp biến có Corr lớn, và xóa đi giá trị ít tác động đến đầu ra
columns_to_remove = []
for col1, col2 in high_corr_pairs.index:
  if col1 == 'TARGET' or col2 == 'TARGET':
    continue
  else:
    # So sánh giá trị trong corr_label và chọn cột có giá trị thấp hơn để xóa
    if corr_label[col1] < corr_label[col2]:
        columns_to_remove.append(col1)
    else:
        columns_to_remove.append(col2)
print(columns_to_remove)

['OBS_60_CNT_SOCIAL_CIRCLE', 'AMT_GOODS_PRICE', 'FLOORSMAX_AVG', 'ELEVATORS_AVG', 'YEARS_BEGINEXPLUATATION_AVG', 'REGION_RATING_CLIENT', 'CNT_FAM_MEMBERS', 'DEF_60_CNT_SOCIAL_CIRCLE', 'LIVE_CITY_NOT_WORK_CITY', 'FLOORSMAX_AVG', 'FLOORSMAX_MODE', 'FLOORSMAX_MODE', 'FLOORSMAX_AVG', 'NAME_FAMILY_STATUS']


In [11]:
data = data.drop(columns = columns_to_remove)
data.shape

(216152, 47)

# **PREDICTION**

In [39]:
#Tách ra những dòng có TARGET = 0
df_zero = data[data['TARGET'] == 0]

# Lấy ngẫu nhiên 50000 dòng từ các dòng có giá trị OUTPUT bằng 0
df_zero = df_zero.sample(n=50000, random_state=42)

#Tách ra những dòng có TARGET = 1
df_one = data[data['TARGET'] == 1]

In [40]:
#Hợp nhất 2 df lại làm một
df_new = pd.concat([df_zero, df_one])

# Xáo trộn ngẫu nhiên các dòng với nhau
data_new = df_new.sample(frac=1, random_state=42).reset_index(drop=True)

In [41]:
#Hàm đánh giá hiệu suất
def print_scores(y_test, y_pred):
  print(classification_report(y_test, y_pred))

#Hàm chia tập train-test và MinMax Scaler
def preprocess_data(data, test_size=0.2, random_state=42):

    # Chia tập train-test
    X = data.drop(columns=['TARGET'], axis=1)
    features = X.columns

    y = data['TARGET']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    #Standard Scaler
    scaler = MinMaxScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    return X_train, X_test, y_train, y_test

In [42]:
X_train, X_test, y_train, y_test = preprocess_data(data_new)

***1.Random Forest***

In [29]:
model_RF = RandomForestClassifier(n_estimators=200, random_state=42, class_weight = 'balanced')
model_RF.fit(X_train, y_train)

y_train_pred = model_RF.predict(X_train)
y_test_pred = model_RF.predict(X_test)

print_scores(y_train, y_train_pred)
print_scores(y_test, y_test_pred)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     39997
           1       1.00      1.00      1.00     14375

    accuracy                           1.00     54372
   macro avg       1.00      1.00      1.00     54372
weighted avg       1.00      1.00      1.00     54372

              precision    recall  f1-score   support

           0       0.75      0.97      0.85     10003
           1       0.60      0.12      0.20      3591

    accuracy                           0.75     13594
   macro avg       0.68      0.54      0.52     13594
weighted avg       0.71      0.75      0.68     13594



***XGBOOST***

In [43]:
model_XGB = xgb.XGBClassifier(n_estimators=200, random_state=42)
model_XGB.fit(X_train, y_train)

y_train_pred = model_XGB.predict(X_train)
y_test_pred = model_XGB.predict(X_test)

print_scores(y_train, y_train_pred)
print_scores(y_test, y_test_pred)

              precision    recall  f1-score   support

           0       0.86      0.98      0.91     39997
           1       0.90      0.55      0.68     14375

    accuracy                           0.86     54372
   macro avg       0.88      0.76      0.80     54372
weighted avg       0.87      0.86      0.85     54372

              precision    recall  f1-score   support

           0       0.77      0.92      0.84     10003
           1       0.52      0.25      0.34      3591

    accuracy                           0.74     13594
   macro avg       0.65      0.58      0.59     13594
weighted avg       0.71      0.74      0.71     13594



***SMOTE vs Under Sampling***

In [32]:
# Áp dụng SMOTE để tăng cường lớp 1
class_counts = y_train.value_counts()
smote = SMOTE(sampling_strategy={1: class_counts[0]}, random_state=42)
X_train_resample, y_train_resample = smote.fit_resample(X_train, y_train)

In [34]:
model_XGB_2 = xgb.XGBClassifier(n_estimators=200, random_state=42)
model_XGB_2.fit(X_train_resample, y_train_resample)

y_train_pred = model_XGB_2.predict(X_train_resample)
y_test_pred = model_XGB_2.predict(X_test)

print_scores(y_train_resample, y_train_pred)
print_scores(y_test, y_test_pred)

              precision    recall  f1-score   support

           0       0.85      0.97      0.91     39997
           1       0.97      0.83      0.89     39997

    accuracy                           0.90     79994
   macro avg       0.91      0.90      0.90     79994
weighted avg       0.91      0.90      0.90     79994

              precision    recall  f1-score   support

           0       0.77      0.91      0.84     10003
           1       0.51      0.26      0.34      3591

    accuracy                           0.74     13594
   macro avg       0.64      0.59      0.59     13594
weighted avg       0.70      0.74      0.71     13594



***Light GBM***

In [46]:
model_light_GBM = LGBMClassifier(random_state=42, n_estimators=200)
model_light_GBM.fit(X_train, y_train)

y_train_pred = model_light_GBM.predict(X_train)
y_test_pred = model_light_GBM.predict(X_test)

print_scores(y_train, y_train_pred)
print_scores(y_test, y_test_pred)

[LightGBM] [Info] Number of positive: 14375, number of negative: 39997
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031238 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2416
[LightGBM] [Info] Number of data points in the train set: 54372, number of used features: 45
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.264382 -> initscore=-1.023314
[LightGBM] [Info] Start training from score -1.023314
              precision    recall  f1-score   support

           0       0.79      0.97      0.87     39997
           1       0.75      0.28      0.41     14375

    accuracy                           0.79     54372
   macro avg       0.77      0.62      0.64     54372
weighted avg       0.78      0.79      0.75     54372

              precision    recall  f1-score   support

           0       0.77      0.95      0.85     10003
      

# **KIỂM THỬ VỚI TẬP TEST**

In [55]:
test = pd.read_csv('/content/drive/MyDrive/Datasets/Home-credit-default-risk/application_test.csv')
test

,SK_ID_CURR,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100001,Cash loans,F,N,Y,0,135000.0,568800.0,20560.5,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,100005,Cash loans,M,N,Y,0,99000.0,222768.0,17370.0,180000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
2,100013,Cash loans,M,Y,Y,0,202500.0,663264.0,69777.0,630000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,1.0,4.0
3,100028,Cash loans,F,N,Y,2,315000.0,1575000.0,49018.5,1575000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,3.0
4,100038,Cash loans,M,Y,N,1,180000.0,625500.0,32067.0,625500.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48739,456221,Cash loans,F,N,Y,0,121500.0,412560.0,17473.5,270000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
48740,456222,Cash loans,F,N,N,2,157500.0,622413.0,31909.5,495000.0,...,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
48741,456223,Cash loans,F,Y,Y,1,202500.0,315000.0,33205.5,315000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,3.0,1.0
48742,456224,Cash loans,M,N,N,0,225000.0,450000.0,25128.0,450000.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0


In [56]:
test = test.dropna()
test = test[~test.isin(['XNA']).any(axis=1)]

In [57]:
test = test.drop(columns = ['ORGANIZATION_TYPE'])
test = label_encoding(test)
test = mapping_cols(test)

In [58]:
cols = data.columns.tolist()
cols.remove('TARGET')
test = test[cols]

In [59]:
test

,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_HOUSING_TYPE,...,FLAG_DOCUMENT_11,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
15,0,1,0,0,180000.0,296280.0,0,4,1,5,...,0,0,0,0,0,0,0,0.0,1.0,3.0
18,0,0,0,1,225000.0,431280.0,0,5,3,0,...,0,0,0,0,0,0,0,0.0,3.0,4.0
73,0,1,0,0,180000.0,67500.0,0,4,1,5,...,0,0,0,0,0,0,0,0.0,0.0,2.0
86,0,1,0,1,225000.0,323460.0,0,4,3,5,...,0,0,0,0,0,0,0,0.0,0.0,3.0
102,0,1,0,0,202500.0,697500.0,0,4,2,0,...,0,0,0,0,0,0,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48628,0,0,0,1,112500.0,252531.0,0,4,3,5,...,0,0,0,0,0,0,0,0.0,0.0,0.0
48670,0,1,0,0,387000.0,350415.0,0,4,3,5,...,0,0,0,0,0,0,0,0.0,1.0,5.0
48682,0,0,0,1,450000.0,790830.0,0,5,3,5,...,0,0,0,0,0,0,0,0.0,0.0,1.0
48707,0,1,0,0,202500.0,225000.0,0,5,3,5,...,0,0,0,0,0,0,0,0.0,1.0,5.0


In [60]:
predict_values = model_XGB.predict(test)

In [63]:
predict_series = pd.Series(predict_values)
predict_series.value_counts()

,count
0,1063
1,676
